In [7]:
%pwd

'/app/heron'

In [3]:
%cd /app
!git clone https://github.com/turingmotors/heron.git

Cloning into 'heron'...
remote: Enumerating objects: 401, done.
remote: Counting objects: 100% (201/201), done.
remote: Compressing objects: 100% (138/138), done.
remote: Total 401 (delta 108), reused 121 (delta 58), pack-reused 200
Receiving objects: 100% (401/401), 10.50 MiB | 9.98 MiB/s, done.
Resolving deltas: 100% (172/172), done.
/app/heron


In [1]:
%cd /app/heron
!pip install -r requirements.txt

/app/heron
  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/d9/92/2d3aecf9f4a192968035880be3e2fc8b48d541c7128f7c936f430d6f96da/accelerate-0.23.0-py3-none-any.whl.metadata
  Using cached accelerate-0.23.0-py3-none-any.whl.metadata (18 kB)
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/09/7e/fd4d6441a541dba61d0acb3c1fd5df53214c2e9033854e837a99dd9e0793/datasets-2.14.5-py3-none-any.whl.metadata
  Using cached datasets-2.14.5-py3-none-any.whl.metadata (19 kB)
  Using cached deepspeed-0.10.3-py3-none-any.whl
  Using cached evaluate-0.4.0-py3-none-any.whl (81 kB)
  Obtaining dependency information for peft from https://files.pythonhosted.org/packages/37/1a/8d20e8704da9fa070eb909265584b960da57be1d833d550c59f50906dc5c/peft-0.5.0-py3-none-any.whl.metadata
  Using cached peft-0.5.0-py3-none-any.whl.metadata (22 kB)
  Obtaining dependency information for protobuf from https://files.pythonhosted.org/packages

In [ ]:
import requests
from PIL import Image

import torch
from transformers import AutoProcessor
from heron.models.git_llm.git_gpt_neox import GitGPTNeoXForCausalLM

device_id = 0

# prepare a pretrained model
model = GitGPTNeoXForCausalLM.from_pretrained(
    'turing-motors/heron-chat-git-ELYZA-fast-7b-v0', 
    torch_dtype=torch.float16,
    ignore_mismatched_sizes=True
)
model = model.half()
model.eval()
model.to(f"cuda:{device_id}")

# prepare a processor
processor = AutoProcessor.from_pretrained('turing-motors/heron-chat-git-ELYZA-fast-7b-v0')

You are using a model of type git_llama to instantiate a model of type git_gpt_neox. This is not supported for all configurations of models and can yield errors.


In [ ]:
# prepare inputs
url = "https://www.barnorama.com/wp-content/uploads/2016/12/03-Confusing-Pictures.jpg"
image = Image.open(requests.get(url, stream=True).raw)
# %cd /app
# image = Image.open("meme-tuna.jpg")

text = f"##human: この写真の面白さを説明してください？\n##gpt: "

# do preprocessing
inputs = processor(
    text,
    image,
    return_tensors="pt",
    truncation=True,
)
inputs = {k: v.to(f"cuda:{device_id}") for k, v in inputs.items()}

# set eos token
eos_token_id_list = [
    processor.tokenizer.pad_token_id,
    processor.tokenizer.eos_token_id,
]

# do inference
with torch.no_grad():
    out = model.generate(**inputs, max_length=256, do_sample=False, temperature=0., eos_token_id=eos_token_id_list)

# print result
print(processor.tokenizer.batch_decode(out)[0])

In [2]:
# heron-chat-git-ja-stablelm-base-7b-v0 を試す
# 一度再起動してから実行する
import requests
from PIL import Image

import torch
from transformers import AutoProcessor
from heron.models.git_llm.git_japanese_stablelm_alpha import GitJapaneseStableLMAlphaForCausalLM

device_id = 0

# prepare a pretrained model
model = GitJapaneseStableLMAlphaForCausalLM.from_pretrained(
    'turing-motors/heron-chat-git-ja-stablelm-base-7b-v0', torch_dtype=torch.float16
)
model.eval()
model.to(f"cuda:{device_id}")

# prepare a processor
processor = AutoProcessor.from_pretrained('turing-motors/heron-chat-git-ja-stablelm-base-7b-v0')

# prepare inputs
url = "https://www.barnorama.com/wp-content/uploads/2016/12/03-Confusing-Pictures.jpg"
image = Image.open(requests.get(url, stream=True).raw)

text = f"##human: これは何の写真ですか？\n##gpt: "

# do preprocessing
inputs = processor(
    text,
    image,
    return_tensors="pt",
    truncation=True,
)
inputs = {k: v.to(f"cuda:{device_id}") for k, v in inputs.items()}

# set eos token
eos_token_id_list = [
    processor.tokenizer.pad_token_id,
    processor.tokenizer.eos_token_id,
]

# do inference
with torch.no_grad():
    out = model.generate(**inputs, max_length=256, do_sample=False, temperature=0., eos_token_id=eos_token_id_list)

# print result
print(processor.tokenizer.batch_decode(out)[0])


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


RecursionError: maximum recursion depth exceeded while getting the str of an object